Finetuning using Google Gemma's Model


In [1]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.7/520.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 13.9 MB/s eta 0:00:00


In [2]:
import transformers, datasets, tokenizers, huggingface_hub

print("Transformers:", transformers.__version__)
print("Datasets:", datasets.__version__)
print("Tokenizers:", tokenizers.__version__)
print("HF Hub:", huggingface_hub.__version__)

Transformers: 5.2.0
Datasets: 4.6.0
Tokenizers: 0.22.2
HF Hub: 1.4.1


In [3]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [4]:
os.environ["HF_token"] = userdata.get('HF_token')

In [44]:
model_id= "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [45]:
tokenizer= AutoTokenizer.from_pretrained(model_id,token=os.environ['HF_token'])
model= AutoModelForCausalLM.from_pretrained(model_id,quantization_config=bnb_config,
                                            device_map={"":0},
                                            token=os.environ['HF_token'])

Loading weights:   0%|          | 0/164 [00:00<?, ?it/s]

In [46]:
text="Quote of albert,"
device = "cuda:0"
inputs = tokenizer(text,return_tensors="pt").to(device)
outputs= model.generate(**inputs, max_new_tokens=30)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote of albert,

"I am a man of action, not of words. I am a man of action, not of words. I am a man of action


In [68]:
os.environ["WANDB_DISABLED"] = "false"

In [47]:
lora_config = LoraConfig(
      r=8,
      target_modules= ["q_proj","o_proj","k_proj","v_proj",
                       "gate_proj","up_proj","down_proj"],
      task_type="CAUSAL_LM",

  )

In [74]:
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")

In [75]:
data['train']['author']

Column(['Oscar Wilde', 'Marilyn Monroe', 'Albert Einstein', 'Frank Zappa', 'Marcus Tullius Cicero', ...])

In [79]:
def formatting_func(example):
    return f"Quote: {example['quote']}\nAuthor: {example['author']}"

In [80]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    formatting_func=formatting_func,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=200,
        learning_rate=2e-4,
        fp16=False,        # safe for T4
        bf16=False,
        report_to="none",
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [81]:
trainer.train()

Step,Training Loss
1,2.662017
2,2.096907
3,2.601519
4,2.861710
5,2.436548
6,2.466273
7,2.819135
8,2.088557
9,2.907265
10,2.223133


/usr/local/lib/python3.12/dist-packages/peft/utils/other.py:1394: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-699f7c25-3f768de97912d2116924f353;8eda5c70-adc5-433c-b44f-b0c511876fe1)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b/resolve/main/config.json.
Access to model google/gemma-2b is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/gemma-2b.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:295: UserWarning: Could not find a config file in google/gemma-2b - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=200, training_loss=1.736454030573368, metrics={'train_runtime': 386.493, 'train_samples_per_second': 2.07, 'train_steps_per_second': 0.517, 'total_flos': 484578235392000.0, 'train_loss': 1.736454030573368})

In [82]:
text="Quote: Oustside of a dog, a book is man's."
device = "cuda:0"
inputs = tokenizer(text,return_tensors="pt").to(device)
outputs= model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: Oustside of a dog, a book is man's.
Author: Mark Twain
